# Applied Computational Economics and Finance

This notebook contains notes and python implementation of some coding examples in "Applied Computational Economics and Finance" by Mario J. Miranda and Paul L. Fackler. 

In [15]:
addpath('C:\Users\barry\Documents\Academics\Quantecon-Notes\functions')
cd 'C:\Users\barry\Documents\Academics\Quantecon-Notes\functions'

## Chapter 3. Nonlinear equations

### Bisection method

Bisection method is a root finding algorithm based on Intermediate Value Theorem. If $f$ is continuous, and $f(a)f(b)<0$, then $f$ must have at least one root in $[a,b]$

Algorithm: WLOG, start with $f(a)<0, f(b)>0$. Compute $f(x) = f(\frac{a+b}{2})$. If $f(x)>0$, then set $b = x$ and continue the iteration. 

Greatest strength: robustness. Guaranteed to compute a root to a precision $\epsilon$ with no more than $\log{((b-a)/\epsilon)}/\log(2)$ iterations. 

### Function iteration:

Function iteration is used to compute a fixed-point, $x=g(x)$ of a function. For root finding, it is equivalent to solve $x=x-f(x)$.

Algorithm: pick an initial guess $x^{0}$. Updating scheme: $x^{(k+1)} = x^{k} - f(x^{k})$

### Newton's Method

Based on the principle of successive linearization: replace nonlinear problem with a sequence of simpler linear problems whose solutions converge to the solution of the nonlinear problem. 

Algorithm: $x^{(k+1)} = x^{k}-[f'(x^k)]^{-1}f(x^k)$

In theory, Newton's method converges if $f$ is continuously differentiable ($C^2$ function) and if the initial value of $x$ is sufficiently close to a root. 

### Quasi-Newton method

Similar to Newton's method but replace $f'$ with something easier to compute. 

1. Secant method: $x^{k+1} = x^k-\frac{x^k-x^{k-1}}{f(x^k)-f(x^{k-1})}f(x^k)$. It requires two starting value.
2. Broyden's method: generates a sequence of vectors $x^k$ and matrices $B^k$ that approximate the root of $f$ and the inveser of the Jacobian matrix. $x^{k+1} = x^k - B^k f(x^k)$ and $B^{k+1} = B^k + ((d^k)-u^k)d^{k^T}B^k)/(d^{k^T}u^k)$ where $u^k = B^k(f(x^{k+1}-f(x^k))$ and $d^k = x^k - x^{k-1}$



**Example: Cournot Duopoly.**

The inverse demand function is $p=q^{-1/\eta}$ and the two firms faces cost function $C_i(q_i)=\frac{1}{2}c_iq_i^2$ for $i=1,2$. The profit of the firm is $\pi_i(q_1, q_2) = P(q_1+q_2)q_i - C(q_i)$ and it solves the FOC: $\frac{\partial \pi_i}{\partial q_i} = 0$

In [17]:
%%file functions/cournot.m

function fval = cournot(q)
    c = [0.6;0.8]; eta = 1.6;
    e = -1/eta;
    fval = sum(q)^e + e*sum(q)^(e-1)*q - diag(c)*q;

Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\functions\cournot.m'.


In [18]:
fsolve(@cournot,[0.2;0.2])


Equation solved.

fsolve completed because the vector of function values is near zero
as measured by the value of the function tolerance, and
the problem appears regular as measured by the gradient.


ans =

    0.8396
    0.6888




### Complementarity problem

For two n-vectors $a$ and $b$, and a function $f$ from $R^n$ to $R^n$, one must find an n-vector s.t. $x_i < a_i \Rightarrow f_i(x)\geq 0$ and $x_i < b_i \Rightarrow f_i(x) \leq 0$ for all $i = 1,2,\cdots n$. Denote the problem to be $CP(f,a,b)$

Root finding is special case with $a_i = -\infty, b_i =\infty$

**Example: simple commodity competitive spatial price equilibrium**

Suppose there are $n$ distinct regions and that excess demand for the commodity is $E_i(p_i)$ of the price $p_i$ in region $i$. In absence of trade, equilibrium is solved by solving $E_i(p_i)=0$ for each $i$. 

Suppose the trade can take place among regions, and the cost of transferring good from $i$ to $j$ is $c_{ij}$. Suppose $x_{ij}$ is the amount shipped from $i$ to $j$ and it cannot exceed capacity $b_{ij}$. Spatial arbitrage profit: $p_j-p_i-c_{ij}$. Equilibrium obtains only if all spatial arbitrage profits are zero. This requires that $0\leq x_{ij}\leq b_{ij}$ and 
1. $x_{ij} > 0 \Rightarrow p_j-p_i-c_{ij} \geq 0$
2. $x_{ij} < b_{ij} \Rightarrow p_j -p_i - c_{ij} \leq 0$

where $p_i$ can be solved using $\sum_k[x_{ki} - x_{ik}] = E_i(p_i)$

Let $f_{ij}(x) = E^{-1}_j(\sum_k[x_{kj} - x_{jk}]) - E^{-1}_i(\sum_k[x_{ki} - x_{ik}]) - c_{ij}$, then $x$ is equilibrium iff it solves $CP(f,0,b)$

### Complementarity Methods

$x$ solves $CP(f,a,b)$ iff it solves $\hat{f}(x) = \min(\max(f(x), a-x), b-x) = 0$, which is a root finding problem.

# Chapter 5: Numerical Integration and Differentiation

## Differentiation
Finite difference: $f'(x_0) \simeq \frac{f(x_0+h)-f(x_0)}{h}$

Two-sided derivative: $f'(x_0) \simeq \frac{f(x_0+h)-f(x_0-h)}{2h}$. It is better as long as the fourth order residual on both sides of $x_0$ are of similar magnitude (prove using Taylor expansion)

In practice, compute TS as $f'(x_0) \simeq \frac{f(x_0+h)-f(x_0-h)}{x_0+h - (x_0-h)}$ so both have same argument. 

Miranda-Fackler: set $h = max(|x_0|,1)\sqrt[3]{\epsilon} \simeq \max(|x_0,1|)\times 6 \times 10^{-6}$, where $\epsilon$ is margion epsilon (for 64-bit machine, $\epsilon = 2^{-52}$).

**Example: Two-sided finite difference for single variable function**

In [25]:
%%file TS_diff.m
function TS = TS_diff(f,x0)
    
    h = max(abs(x0),1)*6.0*1e-6;
    
    TS = (f(x0+h)-f(x0-h))/(2*h);


Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\TS_diff.m'.


In [26]:
TS_diff(@exp, 2)


ans =

    7.3891




**Example: Two-sided finite difference for Jacobian of an arbitrary function**

In [19]:
%%file functions/TS_jac.m
function fjac = TS_jac(f,x)
    
    h = max(abs(x),1)*6.0*1e-6;
    xh1 = x+h;
    xh0 = x-h;
    h = xh1-xh0;
    
    for j =1:length(x)
        xx = x; xx(j) = xh1(j); f1 = feval(f,xx);
        xx(j) = xh0(j); f0 = feval(f,xx);
        fjac(:,j) = (f1-f0)/h(j);
    end
        

Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\functions\TS_jac.m'.


In [20]:
x = [1,2,3,5,1];
TS_jac(@sin, x)


ans =

    0.5403         0         0         0         0
         0   -0.4161         0         0         0
         0         0   -0.9900         0         0
         0         0         0    0.2837         0
         0         0         0         0    0.5403




## Integration